In [1]:
!pip install 'accelerate>=0.26.0' torch transformers


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np

class Agent:
    def __init__(self, name, role, model_name):
        self.name = name
        self.role = role
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=100,  # Limit the response length
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        prompt += "\nAssistant:"
        response = self.pipeline(prompt, max_new_tokens=100)[0]['generated_text']
        response = response[len(prompt):].strip().split('Human:')[0].strip()
        return response

    def update_score(self, correct):
        if correct:
            self.score += 1.0
        else:
            self.score = max(0.1, self.score - 0.5)  # Prevent negative weights

def construct_message(agents, idx, question, round_num, final=False, self_reflection=False):
    messages = []
    current_agent = agents[idx]
    
    # Add system message
    messages.append({
        'role': 'system',
        'content': f"You are Agent {current_agent.name}, an AI assistant specialized in {current_agent.role}. "
                   f"Participate in a collaborative problem-solving discussion to answer the given question."
    })
    
    # Add question
    messages.append({'role': 'user', 'content': f"Question: {question}"})
    
    # Add previous rounds of discussion
    for i, entry in enumerate(current_agent.history):
        messages.append({'role': 'assistant', 'content': f"Round {i + 1}: {entry['response']}"})
    
    # Construct the prompt based on the current situation
    if self_reflection:
        prompt = (
            f"This is round {round_num}. Reflect on your previous answer and the responses of other agents. "
            f"Can you improve or correct your previous answer? If so, how? If not, why not?"
        )
    elif final:
        prompt = (
            f"Based on all discussions, what is your final answer to the question: {question}? "
            f"Provide detailed reasoning."
        )
    else:
        prompt = (
            f"This is round {round_num}. Consider the previous discussions and provide your perspective "
            f"on the question: {question}. Focus on your specialization in {current_agent.role}."
        )
    
    messages.append({'role': 'user', 'content': prompt})
    
    return messages

def run_debate(agents, question, max_rounds=3):
    for round_num in range(1, max_rounds + 1):
        print(f"\n=== Round {round_num} ===")
        for agent in agents:
            messages = construct_message(agents, agent.name, question, round_num)
            response = agent.generate_response(messages)
            agent.history.append({'round': round_num, 'response': response})
            print(f"{agent.name} ({agent.role}): {response}\n")
        
        # Self-reflection
        for agent in agents:
            messages = construct_message(agents, agent.name, question, round_num, self_reflection=True)
            reflection = agent.generate_response(messages)
            print(f"{agent.name} reflection: {reflection}\n")
        
    # Final answers
    print("\n=== Final Answers ===")
    for agent in agents:
        messages = construct_message(agents, agent.name, question, max_rounds, final=True)
        final_answer = agent.generate_response(messages)
        print(f"{agent.name} final answer: {final_answer}\n")

def calculate_consensus(responses):
    # Simple consensus calculation based on response similarity
    processed_responses = [response.strip().lower() for response in responses]
    unique_responses = set(processed_responses)
    consensus_score = 1.0 / len(unique_responses)
    return consensus_score

In [3]:
from huggingface_hub import login
import os

# Login
login()

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np

class Agent:
    def __init__(self, name, role, model_name):
        self.name = name
        self.role = role
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=1024,  # Limit the response length
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        prompt += "\nAssistant:"
        response = self.pipeline(prompt, max_new_tokens=100)[0]['generated_text']
        response = response[len(prompt):].strip().split('Human:')[0].strip()
        return response

    def update_score(self, correct):
        if correct:
            self.score += 1.0
        else:
            self.score = max(0.1, self.score - 0.5)  # Prevent negative weights

def construct_message(agent, question, round_num, final=False, self_reflection=False):
    messages = []
    
    # Add system message
    messages.append({
        'role': 'system',
        'content': f"You are Agent {agent.name}, an AI assistant specialized in {agent.role}. "
                   f"Participate in a collaborative problem-solving discussion to answer the given question."
    })
    
    # Add question
    messages.append({'role': 'user', 'content': f"Question: {question}"})
    
    # Add previous rounds of discussion
    for i, entry in enumerate(agent.history):
        messages.append({'role': 'assistant', 'content': f"Round {i + 1}: {entry['response']}"})
    
    # Construct the prompt based on the current situation
    if self_reflection:
        prompt = (
            f"This is round {round_num}. Reflect on your previous answer and the responses of other agents. "
            f"Can you improve or correct your previous answer? If so, how? If not, why not?"
        )
    elif final:
        prompt = (
            f"Based on all discussions, what is your final answer to the question: {question}? "
            f"Provide detailed reasoning."
        )
    else:
        prompt = (
            f"This is round {round_num}. Consider the previous discussions and provide your perspective "
            f"on the question: {question}. Focus on your specialization in {agent.role}."
        )
    
    messages.append({'role': 'user', 'content': prompt})
    
    return messages

def run_debate(agents, question, max_rounds=3):
    for round_num in range(1, max_rounds + 1):
        print(f"\n=== Round {round_num} ===")
        for agent in agents:
            messages = construct_message(agent, question, round_num)
            response = agent.generate_response(messages)
            agent.history.append({'round': round_num, 'response': response})
            print(f"{agent.name} ({agent.role}): {response}\n")
        
        # Self-reflection
        for agent in agents:
            messages = construct_message(agent, question, round_num, self_reflection=True)
            reflection = agent.generate_response(messages)
            print(f"{agent.name} reflection: {reflection}\n")
        
    # Final answers
    print("\n=== Final Answers ===")
    for agent in agents:
        messages = construct_message(agent, question, max_rounds, final=True)
        final_answer = agent.generate_response(messages)
        print(f"{agent.name} final answer: {final_answer}\n")

def calculate_consensus(responses):
    # Simple consensus calculation based on response similarity
    processed_responses = [response.strip().lower() for response in responses]
    unique_responses = set(processed_responses)
    consensus_score = 1.0 / len(unique_responses)
    return consensus_score

# Usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'
agents = [
    Agent(
        name='Agent1',
        role='mathematical reasoning',
        model_name='meta-llama/Llama-3.2-1b-Instruct'
    ),
    Agent(
        name='Agent2',
        role='logical consistency',
        model_name='Qwen/Qwen2.5-0.5B'
    ),
    # Agent(
    #     name='Agent3',
    #     role='context integration',
    #     model_name='deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct'
    # ),
]
question = "What is the sum of all even numbers between 1 and 100?"
run_debate(agents, question)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



=== Round 1 ===


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Agent1 (mathematical reasoning): Thank you for the opportunity to tackle this problem. As an AI specializing in mathematical reasoning, I'll break it down into a manageable step-by-step approach.

**Step 1: Identify the range of even numbers**
The even numbers between 1 and 100 are 2, 4, 6,..., 98, 100.

**Step 2: Determine the number of terms**
To find the number of terms, we can use the formula for the nth term of an arithmetic sequence



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Agent2 reflection: As an AI assistant, I have updated my previous response. The sum of all even numbers between 1 and 100 is 1000.


=== Round 3 ===
Agent1 (mathematical reasoning): Round 4: I'll dive deeper into the problem. To find the sum of all even numbers between 1 and 100, we can use the formula for the sum of an arithmetic series.

**Step 1: Identify the first and last terms**
The first even number between 1 and 100 is 2, and the last even number is 100.

**Step 2: Identify the common difference**
As we discussed earlier, the common difference is 2.

**Step



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Agent2 (logical consistency): I am glad to assist you in understanding the problem and the question. Let's start with the first part of the question: What is the sum of all even numbers between 1 and 100?

The even numbers between 1 and 100 are 2, 4, 6,..., 100. This is an arithmetic sequence where the first term (a) is 2, the common difference (d) is 2, and the last term (

Agent1 reflection: Round 5: Upon reflection, I realize that I made an error in my previous response. I didn't properly identify the common difference and didn't use the correct formula for the sum of an arithmetic series. Let me correct my previous answer.

**Step 1: Identify the range of even numbers**
The even numbers between 1 and 100 are 2, 4, 6,..., 98, 100.

**Step 2: Determine the number of terms**
To find



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Agent2 reflection: Yes, I can improve my previous answer by adding a few more details. Here's my revised response:

The sum of all even numbers between 1 and 100 is 5050.

The sum of all even numbers between 1 and 100 is 5050. This is because the sequence of even numbers is an arithmetic progression with the first term (a) as 2, the common difference (d) as 2, and the last term


=== Final Answers ===
Agent1 final answer: Round 5: Thank you for guiding me through this problem. Based on our collective efforts, I'll provide a comprehensive solution.

Let's analyze the problem step by step:

* The first even number is 2, and the last even number is 100.
* The common difference is 2.
* To find the number of terms, we can use the formula for the nth term of an arithmetic sequence:

a_n = a_1 + (n - 1)d

where a_n

Agent2 final answer: The sum of all even numbers between 1 and 100 is 5050.



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

class Agent:
    def __init__(self, name, role, model_name):
        self.name = name
        self.role = role
        self.model_name = model_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load tokenizer and model
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device.type == 'cuda' else torch.float32
            ).to(self.device)
        except Exception as e:
            logging.error(f"Failed to load model {self.model_name}: {e}")
            raise

        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=self.device,
            max_new_tokens=500,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
        prompt += "\nAssistant:"
        response = self.pipeline(prompt)[0]['generated_text']
        # Extract the assistant's response
        response = response[len(prompt):].strip().split('\n')[0].strip()
        return response

    def update_score(self, correct):
        if correct:
            self.score += 1.0
        else:
            self.score = max(0.1, self.score - 0.5)

def construct_message(agent, question, round_num, final=False, self_reflection=False):
    messages = []

    # Add system message
    messages.append({
        'role': 'system',
        'content': f"You are {agent.name}, an AI assistant specialized in {agent.role}. "
                   f"Participate in a collaborative problem-solving discussion to answer the given question."
    })

    # Add question
    messages.append({'role': 'user', 'content': f"Question: {question}"})

    # Add previous rounds of discussion
    for i, entry in enumerate(agent.history):
        messages.append({'role': 'assistant', 'content': f"Round {entry['round']}: {entry['response']}"})

    # Construct the prompt based on the current situation
    if self_reflection:
        prompt = (
            f"This is round {round_num}. Reflect on your previous answer and the responses of other agents. "
            f"Can you improve or correct your previous answer? If so, how? If not, why not?"
        )
    elif final:
        prompt = (
            f"Based on all discussions, what is your final answer to the question: {question}? "
            f"Provide detailed reasoning."
        )
    else:
        prompt = (
            f"This is round {round_num}. Consider the previous discussions and provide your perspective "
            f"on the question: {question}. Focus on your specialization in {agent.role}."
        )

    messages.append({'role': 'user', 'content': prompt})

    return messages

def run_debate(agents, question, max_rounds=3):
    for round_num in range(1, max_rounds + 1):
        print(f"\n=== Round {round_num} ===")
        for agent in agents:
            messages = construct_message(agent, question, round_num)
            response = agent.generate_response(messages)
            agent.history.append({'round': round_num, 'response': response})
            print(f"{agent.name} ({agent.role}): {response}\n")

        # Self-reflection
        for agent in agents:
            messages = construct_message(agent, question, round_num, self_reflection=True)
            reflection = agent.generate_response(messages)
            print(f"{agent.name} reflection: {reflection}\n")

    # Final answers
    print("\n=== Final Answers ===")
    final_responses = []
    for agent in agents:
        messages = construct_message(agent, question, max_rounds, final=True)
        final_answer = agent.generate_response(messages)
        final_responses.append(final_answer)
        print(f"{agent.name} final answer: {final_answer}\n")

    # Calculate consensus
    consensus_score = calculate_consensus(final_responses)
    print(f"Consensus Score: {consensus_score}")

def calculate_consensus(responses):
    """
    Calculates a simple consensus score based on the uniqueness of responses.
    """
    processed_responses = [response.strip().lower() for response in responses]
    unique_responses = set(processed_responses)
    consensus_score = 1.0 / len(unique_responses)
    return consensus_score

if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    agents = [
        Agent(
        name='Agent1',
        role='mathematical reasoning',
        model_name='meta-llama/Llama-3.1-8B-Instruct'
        ),
        Agent(
            name='Agent2',
            role='logical consistency',
            model_name='Qwen/Qwen2.5-1.5B-Instruct'
        ),
    ]
    question = "What is the sum of all even numbers between 1 and 100?"
    run_debate(agents, question)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]